### CNN for DDoS intrusion detection

In [1]:
import numpy as np
import pandas as pd

from keras.layers import Conv1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.models import Sequential

from keras import callbacks
from keras.callbacks import CSVLogger

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
import tensorflow as tf
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
tf.logging.set_verbosity(tf.logging.ERROR)

Using TensorFlow backend.


Read and prepare dataset

In [2]:
np.random.seed(667)  # randomize

testdata = pd.read_csv("DDoS2007_spounged.csv", header=None)


#X = traindata.iloc[:,0:42]
#Y = traindata.iloc[:,0]
y_ini = testdata.iloc[:,0]
X_ini = testdata.iloc[:,1:42]

scaler = Normalizer().fit(X_ini)
X_tras = scaler.transform(X_ini) #X Transformed

# reshape input to be [samples, time steps, features]
X = np.reshape(X_tras, (X_tras.shape[0],X_tras.shape[1],1))
y = np.array(y_ini)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

CNN Structure

In [3]:
cnn = Sequential()
cnn.add(Conv1D(filters=16, kernel_size=3, strides=1, padding="same",activation="relu",input_shape=(41, 1)))
cnn.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
cnn.add(MaxPooling1D(pool_size=(2)))
cnn.add(Flatten())
cnn.add(Dense(64, activation="relu"))
cnn.add(Dropout(0.5))
cnn.add(Dense(1, activation="sigmoid"))

cnn.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])

Training (FIT)

In [4]:
cnn.fit(X_train, y_train, epochs=25, validation_data=(X_test, y_test))

Train on 700000 samples, validate on 300001 samples
Epoch 1/25
700000/700000 [==============================] - 547s 781us/step - loss: 0.4718 - acc: 0.7717 - val_loss: 0.4169 - val_acc: 0.8072
Epoch 2/25
700000/700000 [==============================] - 514s 734us/step - loss: 0.4172 - acc: 0.8066 - val_loss: 0.3832 - val_acc: 0.8284
Epoch 3/25
700000/700000 [==============================] - 468s 669us/step - loss: 0.3901 - acc: 0.8249 - val_loss: 0.3554 - val_acc: 0.8430
Epoch 4/25
700000/700000 [==============================] - 480s 685us/step - loss: 0.3710 - acc: 0.8358 - val_loss: 0.3432 - val_acc: 0.8459
Epoch 5/25
700000/700000 [==============================] - 489s 698us/step - loss: 0.3597 - acc: 0.8419 - val_loss: 0.3351 - val_acc: 0.8517
Epoch 6/25
700000/700000 [==============================] - 483s 690us/step - loss: 0.3526 - acc: 0.8446 - val_loss: 0.3256 - val_acc: 0.8575
Epoch 7/25
700000/700000 [==============================] - 487s 695us/step - loss: 0.3482 - acc

Results

In [5]:
y_prob = cnn.predict(X_test)
y_pred = (y_prob > 0.5).astype(np.int)
np.savetxt("cnn.txt", y_pred)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred , average="binary")
precision = precision_score(y_test, y_pred , average="binary")
f1 = f1_score(y_test, y_pred, average="binary")

print("confusion matrix")
print("----------------------------------------------")
print("accuracy %.4f" %accuracy)
print("recall %.4f" %recall)
print("precision %.4f" %precision)
print("f1score %.4f" %f1)
cm = metrics.confusion_matrix(y_test, y_pred)
print("confusion matrix", cm)
print("----------------------------------------------")

confusion matrix
----------------------------------------------
accuracy 0.8775
recall 0.7018
precision 0.9300
f1score 0.7999
confusion matrix [[189781   5530]
 [ 31219  73471]]
----------------------------------------------
